코랩 설정

In [39]:
#from google.colab import drive
#drive.mount('/content/drive')

In [40]:
# 디렉토리가 ~~/notebooks가 되게끔 설정해주세요
# %cd /content/drive/MyDrive/CNN/notebooks

# 경로 설정 및 라이브러리/모듈 불러오기

In [41]:
import os

# 디렉토리가 ~~/notebooks가 되게끔 설정해주세요
# os.chdir()
print(os.getcwd())

/media/nvme1/seungbeom/[0227]CNN_FirstnameLASTNAME/[0227]CNN_FirstnameLASTNAME/notebooks


In [42]:
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
data_dir = os.path.join(ROOT_DIR, "data")
weights_dir = os.path.join(ROOT_DIR, "weights")
results_dir = os.path.join(ROOT_DIR, "results")
config_path = os.path.join(ROOT_DIR, "configs", "config.yaml")
metadata_path = os.path.join(ROOT_DIR, "data", "metadata.csv")

sys.path.append(os.path.join(ROOT_DIR, "models"))
sys.path.append(os.path.join(ROOT_DIR, "scripts"))
os.makedirs(weights_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

In [43]:

from CustomDataset import CustomImageDataset
from resnet import ResNet50

import pandas as pd
import numpy as np

import yaml

import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

# 하이퍼파라미터 및 기본 설정들

**config.yaml**에 지정해줬던 하이퍼파라미터들을 가져옵시다.

In [44]:
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

batch_size, lr, epochs \
    = config["no_augmentation"]["hyperparameters"]["batch_size"], \
      float(config["no_augmentation"]["hyperparameters"]["learning_rate"]), \
      config["no_augmentation"]["hyperparameters"]["num_epochs"]

In [45]:
# 가속을 위해 device로 GPU로 설정해줍니다
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [46]:
# num_workers는 dataset의 데이터를 gpu로 전송할 때 필요한 전처리를 수행할 때 사용하는 subprocess의 수입니다
# 참고: https://velog.io/@seokjin1013/PyTorch-numworkers%EC%97%90-%EA%B4%80%ED%95%98%EC%97%AC
num_workers = min(4, os.cpu_count() // 2)
print(num_workers)

4


# train, val, test DataLoader 초기화

In [61]:
# MetaData.csv를 로드해주고, train, val, test로 나눠줍니다
meta_data = pd.read_csv(os.path.join(data_dir, 'metadata.csv'))

# train : val : test = 6 : 2 : 2
train, val_test = train_test_split(meta_data, train_size=0.6, random_state=2025)
val, test = train_test_split(val_test, train_size=0.5, random_state=2025)
print(train)
# CustomDataset.py에서 가져온 CustomImageDataset 클래스에 train, val, test 데이터프레임을 패스하여 데이터셋을 만들어주세요
train_dataset = CustomImageDataset(data_dir, train)
val_dataset   = CustomImageDataset(data_dir, val)
test_dataset  = CustomImageDataset(data_dir, test)

train_loader = DataLoader(train_dataset, batch_size = 8, shuffle=True, num_workers=num_workers)
val_loader   = DataLoader(val_dataset, batch_size = 8, shuffle=True, num_workers=num_workers)
test_loader  = DataLoader(test_dataset, batch_size = 8, shuffle=True, num_workers=num_workers)

In [ ]:
train_dataset.__getitem__(565)

# 모델 초기화, 손실함수 및 optimizer 지정

In [62]:
# model을 선언하고 GPU에 올려줍니다
model = ResNet50()
model.to(device)

/media/nvme1/fsl/lib/python3.12/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


ResNet50(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResBottleNeck(
      (conv1): Conv2d(1, 0, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(0, 0, kernel_size=(3, 3), stride=(64, 64), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(0, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(1, 64, kernel_size=(1, 1), stride=(64, 64)

In [63]:
# 손실함수와 optimizer를 정의합니다 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train

In [65]:
# Best Validation Loss를 양의 무한대로 초기화
best_loss = float("inf")

# 모든 epoch에서의 loss와 accuracy들을 저장할 리스트들
train_loss_list = []
valid_loss_list = []
train_accuracy_list = []
valid_accuracy_list = []
print(len(train_loader))
for curr_epoch in range(epochs): # iterate over all EPOCHS
    running_loss, total_correct, total_samples = 0, 0, 0 # 현재 epoch에서의 train_loss와 train_accuracy들을 계산하기 위한 변수들
    #print(curr_epoch)
    for batch_idx, (img, label) in enumerate(train_loader): # iterate over all BATCHES
        # GPU에 올라가는 건 모델 뿐 아니라 데이터도 같이 올라가야합니다!
        img, label = img.to(device), label.to(device)

        # Forward Propagation
        pred = model(img)
        
        # 손실함수 계산
        loss = criterion(pred, label) # 구현 상의 차이로 label 텐서에 조작을 가해 모양을 맞춰줘야하는 경우가 생길 수 있습니다 ( 힌트 : tensor.view() )

        # 누적된 기울기 초기화 및 역전파
        optimizer.zero_grad()
        loss.backward()

        # 파라미터 업데이트
        optimizer.step()

        # 해당 Batch에서의 loss를 running_loss 변수에 누적
        running_loss += loss.item()
        
        # Accuracy 계산
        predicted = (torch.argmax(pred,dim=1))
        label = label.float()
        
        total_correct += (predicted == label).sum().item()
        
        total_samples += label.size(0)

    # 모든 Batch를 순회한 이후 최종적인 train_loss, train_accuracy를 계산해줍니다
    train_loss = running_loss / len(train_loader)
    train_accuracy = total_correct / total_samples

    
    # Validation 시작
    model.eval()
    
    with torch.no_grad():  # 학습 중이 아니므로 역전파를 위한 gradient 연산은 필요하지 않습니다
        running_loss, total_correct, total_samples = 0, 0, 0 # 현재 epoch에서의 val_loss와 val_accuracy들을 계산하기 위한 변수들
        
        for img, label in val_loader: # iterate over all BATCHES
            # GPU에 올라가는 건 모델 뿐 아니라 데이터도 같이 올라가야합니다!
            img, label = img.to(device), label.to(device)

            # Forward Propagation
            pred = model(img)

            # 손실함수 계산
            loss = criterion(pred, label) # 구현 상의 차이로 label 텐서에 조작을 가해 모양을 맞춰줘야하는 경우가 생길 수 있습니다 ( 힌트 : tensor.view() )

            # 해당 Batch에서의 loss를 running_loss 변수에 누적
            running_loss += loss.item()

            # 예측값 및 Accuracy 계산
            predicted = (torch.argmax(pred,dim=1))
            label = label.float()
            total_correct += (predicted == label).sum().item()
            total_samples += label.size(0)

        # 모든 Batch를 순회한 이후 최종적인 train_loss, train_accuracy를 계산해줍니다
        val_loss = running_loss / len(val_loader)
        val_accuracy = total_correct / total_samples


    model.train()  # 모델을 다시 학습모드로 설정합니다

    # 현재 epoch에서의 결과들 출력
    print(f"Epoch [{curr_epoch + 1}/{epochs}] | "
          f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

    # 리스트에 저장
    train_loss_list.append(train_loss)
    valid_loss_list.append(val_loss)
    train_accuracy_list.append(train_accuracy)
    valid_accuracy_list.append(val_accuracy)
    
    # 가장 작은 validation loss를 가졌을 때의 모델 파라미터를 "best_model_no_augmentation.pth"로 저장해줍니다.
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), os.path.join(weights_dir, "best_model_no_augmentation.pth"))


225


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/media/nvme1/fsl/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 555

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/media/nvme1/fsl/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^

# Train 결과 시각화 및 그래프 저장

In [ ]:
# Train Loss & Validation Loss 그래프 시각화 및 저장
plt.figure(figsize=(12, 5))
plt.plot(range(1, epochs + 1), train_loss_list, label='Train Loss', marker='o', linestyle='-')
plt.plot(range(1, epochs + 1), valid_loss_list, label='Validation Loss', marker='s', linestyle='--')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train & Validation Loss - No Augmentation')
plt.legend()
plt.savefig(os.path.join(results_dir, "train_validation_loss_no_augmentation.png"))
plt.show()
plt.close()

# Train Accuracy & Validation Accuracy 그래프 시각화 및 저장
plt.figure(figsize=(12, 5))
plt.plot(range(1, epochs + 1), train_accuracy_list, label='Train Accuracy', marker='o', linestyle='-')
plt.plot(range(1, epochs + 1), valid_accuracy_list, label='Validation Accuracy', marker='s', linestyle='--')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Train & Validation Accuracy - No Augmentation')
plt.legend()
plt.savefig(os.path.join(results_dir, "train_validation_accuracy_no_augmentation.png"))
plt.show()
plt.close()


# Test 및 Test 결과 시각화

In [ ]:
# 우선 ResNet50 모델 하나를 초기화 해줍니다
trained_model = ResNet50()
# 아까 저장해둔 weight를 로드해줍니다
trained_model.load_state_dict(torch.load(os.path.join(weights_dir, "best_model_no_augmentation.pth"), weights_only=True))
# 모델을 evaluation 모드로 설정합니다
trained_model.eval()
trained_model.to(device)

In [ ]:
all_labels = []
all_predictions = []

with torch.no_grad():  # 학습 중이 아니므로 역전파를 위한 gradient 연산은 필요하지 않습니다
    for image, label in test_loader:
        # GPU에 올라가는 건 모델 뿐 아니라 데이터도 같이 올라가야합니다!
        image, label = image.to(device), label.to(device)

        # Forward Propagation
        pred = trained_model(image)

        # 예측값 계산
        predicted = (torch.argmax(pred,dim=1))

        # 리스트에 저장
        all_labels.append(label)
        all_predictions.append(predicted)

all_labels = torch.cat(all_labels).cpu().numpy().tolist()
all_predictions = torch.cat(all_predictions).cpu().numpy().tolist()

In [ ]:
# Accuracy 계산 및 Confusion Matrix 계산
accuracy = accuracy_score(all_labels, all_predictions)
cm = confusion_matrix(all_labels, all_predictions)

print("Accuracy: {:.2f}%".format(accuracy * 100))

# Confusion Matrix 시각화 및 저장

fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title(f'Confusion Matrix & Accuracy: {accuracy * 100:.2f}% - No Augmentation')
plt.savefig(os.path.join(results_dir, "test_accuracy_no_augmentation.png"))
plt.show()
plt.close()